# 从获得期货每日价格并导入Database
流程：

（1） 制作网址
（2）将网址传入，获得页码数据反馈
（3）将每一页的数据拔下来

In [51]:
from lxml import etree
from lxml import html
import re
import sqlite3
import requests

In [52]:
def userEnter(contract, startdate, enddate):

    url = "http://vip.stock.finance.sina.com.cn/q/view/vFutures_History.php?jys=czce&pz=MA&hy=MA1909&breed=" +contract + "&type=inner&start=" + startdate +"&end=" +enddate
    return url

In [53]:
url = userEnter('AG2012','2019-01-01','2020-05-26')
print(url)

http://vip.stock.finance.sina.com.cn/q/view/vFutures_History.php?jys=czce&pz=MA&hy=MA1909&breed=AG2012&type=inner&start=2019-01-01&end=2020-05-26


In [54]:
url

'http://vip.stock.finance.sina.com.cn/q/view/vFutures_History.php?jys=czce&pz=MA&hy=MA1909&breed=AG2012&type=inner&start=2019-01-01&end=2020-05-26'

## 设置一个url list

In [55]:
url_prefix = 'http://vip.stock.finance.sina.com.cn/'

In [56]:
url_list = []
url_list.append(url)

In [57]:
print(url_list)

['http://vip.stock.finance.sina.com.cn/q/view/vFutures_History.php?jys=czce&pz=MA&hy=MA1909&breed=AG2012&type=inner&start=2019-01-01&end=2020-05-26']


## 找到有多少个link

In [58]:
response = requests.get(url)
response.encoding = 'gb2312'
# print(response.text)
with open('sinatest.html', 'w', encoding='gb2312')as f:
    f.write(response.text)

with open('sinatest.html', 'r')as a:
    contend = a.read()
tree = etree.HTML(contend)
webs = tree.xpath("//div[@class = 'historyList']//@href")    
webs = webs[1:(len(webs)-1)]

for each in webs:
    next_url = url_prefix + each
    url_list.append(next_url)

In [59]:
url_list

['http://vip.stock.finance.sina.com.cn/q/view/vFutures_History.php?jys=czce&pz=MA&hy=MA1909&breed=AG2012&type=inner&start=2019-01-01&end=2020-05-26',
 'http://vip.stock.finance.sina.com.cn//q//view/vFutures_History.php?page=2&breed=AG2012&start=2019-01-01&end=2020-05-26&jys=czce&pz=MA&hy=MA1909&type=inner&name=大豆1109',
 'http://vip.stock.finance.sina.com.cn//q//view/vFutures_History.php?page=3&breed=AG2012&start=2019-01-01&end=2020-05-26&jys=czce&pz=MA&hy=MA1909&type=inner&name=大豆1109']

In [ ]:
response = requests.get(url)
response.encoding = 'gb2312'
# print(response.text)
with open('sinatest.html', 'w', encoding='gb2312')as f:
    f.write(response.text)

with open('sinatest.html', 'r')as a:
    contend = a.read()

tree = etree.HTML(contend)
for link in tree.xpath("//div[@class = 'historyList']//@href"):
    print(link)
    #print('1')
    url_list = tree.xpath("//div[@class = 'historyList']//@href")
    numpage = re.findall('page=(\d)', url_list[len(url_list) - 2]).pop()
    # print(numpage)
num = 1
for each in range(int(numpage) - 1):
    num += 1

In [ ]:
def findAll_url(url):
    response = requests.get(url)

    response.encoding = 'gb2312'
    # print(response.text)
    with open('sinatest.html', 'w', encoding='gb2312')as f:
        f.write(response.text)

    with open('sinatest.html', 'r')as a:
        contend = a.read()

    tree = etree.HTML(contend)
    for link in tree.xpath("//div[@class = 'historyList']//@href"):
        print(link)
        #print('1')
        url_list = tree.xpath("//div[@class = 'historyList']//@href")
        numpage = re.findall('page=(\d)', url_list[len(url_list) - 2]).pop()
        # print(numpage)
    num = 1
    for each in range(int(numpage) - 1):
        num += 1
        #print(num)

In [60]:
def enter_intoDB(url):

    response = requests.get(url)

    # 显示乱码
    # print(response.text)

    response.encoding = 'GB18030'
    #print(response.text)
    with open('sinatest.html', 'w', encoding='GB18030')as f:
        f.write(response.text)

    with open('sinatest.html', 'r')as a:
        contend = a.read()
        # print(contend)

    tree = etree.HTML(contend)

    tablerow = tree.xpath("//div[@class = 'historyList']")[0]

    pattern_atribute = re.compile('<.*?>')
    pattern_atribute2 = re.compile('(#\d+)')

    text = str(etree.tostring(tablerow))
    text2 = pattern_atribute.sub('', text)
    text2 = pattern_atribute2.sub('', text2)
    #print(text2)
    contractName = re.findall('\(([A-Z]+\d+)\)', text2)[0]
    # print(re.findall('t(\d+\-\d+\-\d+)',text2))
    # print(re.findall('t(\d+)',text2))
    dateList = re.findall('t(\d+\-\d+\-\d+)', text2)
    # print(dateList)
    numOfDate = 0
    for each in dateList:
        numOfDate += 1
    print(numOfDate)

    dataOfTarget = re.findall('t(\d+)', text2)
    dataOfTarget.reverse()
    numOfData = 0
    for i in dataOfTarget:
        numOfData += 1
    print(numOfData)

    for each in dateList:
        enter_list = []
        IDcode = contractName + each
        enter_list.append(IDcode)
        enter_list.append(contractName)
        enter_list.append(each)

        # ['MA1909','2019-04-09']
        order = 0
        for i in range(6):
            order += 1
            if order < 2:
                dataOfTarget.pop()
            else:
                enter_list.append(dataOfTarget.pop())
        
        print(enter_list)

In [61]:
for i in url_list:
    enter_intoDB(i)

40
240
['AG20122020-05-26', 'AG2012', '2020-05-26', '4288', '4195', '4319', '4180', '1012424']
['AG20122020-05-25', 'AG2012', '2020-05-25', '4167', '4198', '4250', '4153', '1122137']
['AG20122020-05-22', 'AG2012', '2020-05-22', '4150', '4226', '4247', '4100', '1314484']
['AG20122020-05-21', 'AG2012', '2020-05-21', '4214', '4289', '4326', '4190', '965581']
['AG20122020-05-20', 'AG2012', '2020-05-20', '4329', '4203', '4346', '4195', '1109965']
['AG20122020-05-19', 'AG2012', '2020-05-19', '4190', '4243', '4261', '4138', '937585']
['AG20122020-05-18', 'AG2012', '2020-05-18', '4248', '4090', '4248', '4030', '708216']
['AG20122020-05-15', 'AG2012', '2020-05-15', '4005', '3825', '4034', '3811', '622050']
['AG20122020-05-14', 'AG2012', '2020-05-14', '3831', '3843', '3858', '3775', '266206']
['AG20122020-05-13', 'AG2012', '2020-05-13', '3821', '3811', '3844', '3789', '228065']
['AG20122020-05-12', 'AG2012', '2020-05-12', '3795', '3811', '3831', '3771', '198351']
['AG20122020-05-11', 'AG2012', '

In [76]:
from lxml import etree
from lxml import html
import re
import sqlite3
import requests


response = requests.get('http://vip.stock.finance.sina.com.cn/q/view/vFutures_History.php?jys=czce&pz=MA&hy=MA1909&breed=AG2006&type=inner&start=2019-01-01&end=2020-07-01')
response.encoding = 'gb18030'
# print(response.text)
with open('sinatest.html', 'w', encoding='gb18030')as f:
    f.write(response.text)

with open('sinatest.html', 'r')as a:
    contend = a.read()

tree = etree.HTML(contend)

web_with_last_page = tree.xpath("//div[@class = 'historyList']//@href")[-2]
numpage = int(re.findall('page=(\d)',web_with_last_page)[0])

for i in range(2,numpage+1):
    page_number = 'page=' + str(i)
    new_url = re.sub('page=(\d)',page_number,web_with_last_page,)
    next_url = url_prefix + new_url
    url_list.append(next_url)


In [66]:
tree.xpath("//div[@class = 'historyList']//@href")

['https://finance.sina.com.cn/futures/quotes/AG2006.shtml',
 '/q//view/vFutures_History.php?page=2&breed=AG2006&start=2019-01-01&end=2020-07-01&jys=czce&pz=MA&hy=MA1909&type=inner&name=大豆1109',
 '/q//view/vFutures_History.php?page=6&breed=AG2006&start=2019-01-01&end=2020-07-01&jys=czce&pz=MA&hy=MA1909&type=inner&name=大豆1109',
 '#']

In [67]:
test_web = web_with_last_page

In [75]:
for i in range(2,numpage+1):
    page_number = 'page=' + str(i)
    new_url = re.sub('page=(\d)',page_number,web_with_last_page,)
    next_url = url_prefix + new_url
    url_list.append(next_url)

http://vip.stock.finance.sina.com.cn//q//view/vFutures_History.php?page=2&breed=AG2006&start=2019-01-01&end=2020-07-01&jys=czce&pz=MA&hy=MA1909&type=inner&name=大豆1109
http://vip.stock.finance.sina.com.cn//q//view/vFutures_History.php?page=3&breed=AG2006&start=2019-01-01&end=2020-07-01&jys=czce&pz=MA&hy=MA1909&type=inner&name=大豆1109
http://vip.stock.finance.sina.com.cn//q//view/vFutures_History.php?page=4&breed=AG2006&start=2019-01-01&end=2020-07-01&jys=czce&pz=MA&hy=MA1909&type=inner&name=大豆1109
http://vip.stock.finance.sina.com.cn//q//view/vFutures_History.php?page=5&breed=AG2006&start=2019-01-01&end=2020-07-01&jys=czce&pz=MA&hy=MA1909&type=inner&name=大豆1109
http://vip.stock.finance.sina.com.cn//q//view/vFutures_History.php?page=6&breed=AG2006&start=2019-01-01&end=2020-07-01&jys=czce&pz=MA&hy=MA1909&type=inner&name=大豆1109


In [70]:
sth = re.sub('page=(\d)','page=3',web_with_last_page,)


'/q//view/vFutures_History.php?page=3&breed=AG2006&start=2019-01-01&end=2020-07-01&jys=czce&pz=MA&hy=MA1909&type=inner&name=大豆1109'